In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
import scheme_mar2023 as scheme
import mytools

In [ ]:
#picarro_data_path = r'g:\1_Data1\93_BaikalJun2023Sea\scouter\2023'
picarro_data_path = r'G:\1_Data1\97_BaikalJun2024Sea\origin\2024'
#picarro_data_path = r'G:\1_Data1\98_BaikalJul2024Sea\origin'
#picarro_data_path = r'G:\1_Data1\!AMK96\picarro'
files = glob.glob(picarro_data_path+r'\*\*\*.dat')
len(files)

In [ ]:
df = pd.DataFrame()
for file in files:
    dft = pd.read_csv(file,
                      sep='\s+',
                      usecols=lambda x: x.lower() in ['date', 'time', 'ch4_dry', 'co2_dry'],
                      parse_dates=[['DATE', 'TIME']],
                      # nrows=1000,
                      )
    df = pd.concat([df, dft], ignore_index=True)
    print(file)
    

In [ ]:
# df.reset_index(drop=True)
df.sort_values(by='DATE_TIME', ascending=True, axis='index', inplace=True)
df.info()

In [ ]:
df = df.resample('10S', on='DATE_TIME',).mean()
df

In [ ]:
df['DATE_TIME_sec'] = df.index.astype('int64')/10**9
df['DATE_TIME_sec'] = df['DATE_TIME_sec'].astype('int')
df['DATE_TIME_sec'] = df['DATE_TIME_sec'] + 8*60*60 + 2*60 + 50
df['DateTime'] = pd.to_datetime(df['DATE_TIME_sec'], unit='s')
df

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x='DateTime', y='CH4_dry', data=df)
ax.legend()

In [ ]:
df.rename(columns=scheme.NAME_CONV_BAS, inplace=True)
df.sort_values(by='DateTime', inplace=True)
df.reset_index(inplace=True, drop=True)
df.tail(2)

In [ ]:
df.drop(['DATE_TIME_sec'], axis='columns', inplace=True)
df

In [ ]:
df['vCO2'] = df['vCO2'] * scheme.CO2PicarroScaut_cor[0] + scheme.CO2PicarroScaut_cor[1]
df['vCH4'] = df['vCH4'] * scheme.CH4PicarroScaut_cor[0] + scheme.CH4PicarroScaut_cor[1]


In [ ]:
df.to_csv(picarro_data_path+r'\..\picarroCor.dat', sep='\t')

In [ ]:
df